#### Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:

* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file mlflow_on_aws.md to create a new AWS account and launch the tracking server.

In [4]:
import os
import configparser
conf = configparser.RawConfigParser()
conf.read('/mnt/c/Users/aruna/.aws/credentials')
conf.sections()

aws_access_key_id = conf.get('default','aws_access_key_id')
aws_secret_access_key = conf.get('default','aws_secret_access_key')

os.environ['ACCESS_KEY_ID'] = aws_access_key_id
os.environ['ACCESS_SECRET_KEY'] = aws_secret_access_key

In [5]:
import mlflow
import os

#os.environ["AWS_PROFILE"] = ""

TRACKING_SERVER_HOST = "ec2-13-232-19-180.ap-south-1.compute.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [6]:
print(f"tracking uri : '{mlflow.get_tracking_uri()}'")

tracking uri : 'http://ec2-13-232-19-180.ap-south-1.compute.amazonaws.com:5000'


In [7]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://mlfow-artifacts-remote-arun/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2022/06/04 20:45:03 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
/home/arunabha/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


default artifacts URI: 's3://mlfow-artifacts-remote-arun/1/90029cf0be984045894dc8adabee1d04/artifacts'


In [9]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://mlfow-artifacts-remote-arun/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://mlfow-artifacts-remote-arun/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

In [10]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [11]:
client.list_registered_models()

[]

In [12]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
print(run_id)

90029cf0be984045894dc8adabee1d04


In [13]:
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2022/06/04 20:46:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1654355817462, current_stage='None', description='', last_updated_timestamp=1654355817462, name='iris-classifier', run_id='90029cf0be984045894dc8adabee1d04', run_link='', source='s3://mlfow-artifacts-remote-arun/1/90029cf0be984045894dc8adabee1d04/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [14]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1654355817217, description='', last_updated_timestamp=1654355817462, latest_versions=[<ModelVersion: creation_timestamp=1654355817462, current_stage='None', description='', last_updated_timestamp=1654355817462, name='iris-classifier', run_id='90029cf0be984045894dc8adabee1d04', run_link='', source='s3://mlfow-artifacts-remote-arun/1/90029cf0be984045894dc8adabee1d04/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]